In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.offline as po
import os 

This notebook contains exploration of all the raw data and will conclude with proposals for data transformations and wrangling.

In [2]:
df = pd.read_csv('_raw/case1Data.txt')

In [13]:
# NA values are by default strings, since there is a space in front
df = df.replace(' NaN', np.nan)


In [19]:
# only 4 out of 100 cols are complete.
# the rest contain NA's and require a decision on what to do
df.loc[:,df.isna().sum(axis=0) == 0].columns

Index(['y', ' C_ 2', ' C_ 3', ' C_ 4', ' C_ 5'], dtype='object')